In [ ]:
# | default_exp routes.instance_config


In [ ]:
# | export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

import domolibrary.routes.user as user_routes


In [ ]:
# | export
async def get_allowlist(
    auth: dmda.DomoFullAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
) -> rgd.ResponseGetData:

    if auth.__class__.__name__ != "DomoFullAuth":
        raise dmda.InvalidAuthTypeError(
            function_name="get_allowlist",
            domo_instance=auth.domo_instance,
            required_auth_type=dmda.DomoFullAuth,
        )

    url = f"https://{auth.domo_instance}.domo.com/admin/companysettings/whitelist"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        headers={"accept": "*/*"},
        session=session,
        debug_api=debug_api,
        is_follow_redirects=True,
        return_raw=return_raw,
    )

    return res

#### sample implementation of get_allowlist with the wrong auth type


In [ ]:
import os
import requests

auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    await get_allowlist(auth=auth)
except dmda.InvalidAuthTypeError as e:
    print(e)

get_allowlist: This API rquires DomoFullAuth at domo-dojo


#### sample implementation of get_allowlist with the correct auth type


In [ ]:
import os
import requests

auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

res = await get_allowlist(auth=auth)

res

ResponseGetData(status=200, response={'addresses': ['0.0.0.0/0', '192.168.0.1', '20.169.16.154']}, is_success=True)

In [ ]:
# | export
class Allowlist_UnableToUpdate(de.DomoError):
    def __init__(
        self, 
        status: int, reason: str, domo_instance:str, function_name: str = "update_allowlist"
    ):

        super().__init__(
            function_name=function_name,
            status=status,
            message=f"unable to update allowlist: {reason}",
            domo_instance=domo_instance,
        )

In [ ]:
# | export


async def set_allowlist(
    auth: dmda.DomoAuth,
    ip_address_ls: list[str],
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None
) -> rgd.ResponseGetData:
    """companysettings/whitelist API only allows users to SET the allowlist does not allow INSERT or UPDATE"""

    url = f"https://{auth.domo_instance}.domo.com/admin/companysettings/whitelist"

    body = {"addresses": ip_address_ls}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        is_follow_redirects=True,
        return_raw=return_raw,
        session = session,
        headers={"accept": "text/plain"},
    )
    if not res.is_success:
        raise Allowlist_UnableToUpdate(
            status=res.status, reason=res.response, domo_instance=auth.domo_instance
        )

    if res.is_success:
        res.response = f"allow list updated from {res.response}"

    return res

In [ ]:
import os

ip_address_ls = ["0.0.0.0/0"]

auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

res = await set_allowlist(auth=auth, ip_address_ls=ip_address_ls)
res

ResponseGetData(status=200, response='allow list updated from 20.169.16.154', is_success=True)

In [ ]:
# | export
async def set_authorized_domains(
    auth: dmda.DomoAuth,
    authorized_domain_ls: [str],
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/authorized-domains"

    body = {"name": "authorized-domains", "value": authorized_domain_ls}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        log_results=log_results,
        debug_api=debug_api,
        session=session,
    )

    return res


In [ ]:
# | export

class GetDomains_NotFound(de.DomoError):
    def __init__(self, status, message, domo_instance):
        super().__init__(status = status, message = message, domo_instance = domo_instance)



In [ ]:
# | export
async def get_authorized_domains(
    auth: dmda.DomoAuth,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/authorized-domains"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    # domo raises a 404 error even if the success is valid but there are no approved domains
    if res.status == 404 and res.response == 'Not Found':
        res_test = await user_routes.get_all_users(auth = auth)

        if not res_test.is_success:
            raise GetDomains_NotFound(
                domo_instance=auth.domo_instance, status=res.status, message=res.response)
        
        if res_test.is_success:
            res.status = 200
            res.is_success = True
            res.response = []
        
        return res
    
    res.response = [domain.strip() for domain in res.response.get('value').split(',')]
    return res


#### sample implementation of get_authorized_domains

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance= 'domo-domo',
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    res = await get_authorized_domains(auth=token_auth, debug_api=False)
    print(res)

except Exception as e:
    print(e)
    

ResponseGetData(status=200, response=[], is_success=True)


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
